In [1]:
# import dependencies
import pandas as pd
import numpy as np
import os

In [2]:
# Pull in original cancer data csvs from CDC and convert to dataframes
# CSV pulled from https://wonder.cdc.gov/
lung_df1 = pd.read_csv(os.path.join("Resources", "CDC_Cancer1.csv"))
lung_df2 = pd.read_csv(os.path.join("Resources", "CDC_Cancer2.csv"))
county_df = pd.read_csv(os.path.join("Resources", "CDC_Cancer_by_County.csv"))

# Pull in original poverty CSV from FRED and convert to dataframe
# CSV pulled from https://geofred.stlouisfed.org/map/?th=pubugn&cc=5&rc=false&im=fractile&sb&lng=-49.6&lat=45.5&zm=2&sl&sv&am=Average&at=Not%20Seasonally%20Adjusted,%20Annual,%20Percent&dt=2015-01-01&fq=Annual&rt=county&sti=150203&un=lin
povDF = pd.read_csv(os.path.join('Resources','Poverty%ByUSCounty.csv'))


## Clean CDC Lung Cancer by State Data

In [3]:
# Combine lung dataframes into single dataframe
lung_df = lung_df1.append(lung_df2, ignore_index=True)

# Delete unnecessary columns
lung_df = lung_df.drop(["Notes", "State Code", "Year Code", "Sex Code", "Race Code", "Age Group Code"], axis=1)

# Check to see if there are any null values in dataframe
lung_df.isnull().sum()

State         102
Year          102
Sex           102
Race          102
Age Group     102
Deaths        102
Population    102
Crude Rate    102
dtype: int64

In [4]:
# Drop rows that have nothing but null values
lung_df = lung_df.dropna(how="all")
# Verify there are no null values left
lung_df.isnull().sum()

State         0
Year          0
Sex           0
Race          0
Age Group     0
Deaths        0
Population    0
Crude Rate    0
dtype: int64

## Clean CDC Lung Cancer by County Data

In [5]:
# Delete unnecessary columns
county_df = county_df.drop(["Notes", "County Code", "Year Code"], axis =1)

# Check to see if there are any null values in dataframe
county_df.isnull().sum()

County        81
Year          81
Deaths        81
Population    81
Crude Rate    81
dtype: int64

In [6]:
# Drop rows that have nothing but null values
county_df = county_df.dropna(how="all")
# Verify there are no null values left
county_df.isnull().sum()

County        0
Year          0
Deaths        0
Population    0
Crude Rate    0
dtype: int64

In [7]:
# For county dataframe, remove unreliable rows and rows with missing information
county_df = county_df.loc[county_df["Crude Rate"] != "Unreliable", :]
county_df = county_df.loc[county_df["Deaths"] != "Missing", :]

In [8]:
# Export cancer cleaned cancer dataframes to csv files

lung_df.to_csv(os.path.join('Output','Cancer_by_State_Cleaned.csv'))
county_df.to_csv(os.path.join('Output','Cancer_by_County_Cleaned.csv'))

## Clean FRED Poverty Data by County

In [9]:
# Check to see if there are any null values in dataframe
povDF.isnull().sum()

County    0
State     0
2012      0
2013      0
2014      2
2015      6
2016      6
2017      6
dtype: int64

In [10]:
# Drop rows that have null values
povDF = povDF.dropna(how="any")
# Verify there are no null values left
povDF.isnull().sum()

County    0
State     0
2012      0
2013      0
2014      0
2015      0
2016      0
2017      0
dtype: int64

In [11]:
# Examine data
povDF = povDF.sort_values("County")
povDF.head()

,County,State,2012,2013,2014,2015,2016,2017
4632,"Abbeville County, SC",SC,20.8,21.6,23.1,22.5,21.6,22.7
4678,"Abbeville County, SC",SC,20.8,21.6,23.1,22.5,21.6,22.7
2290,"Acadia Parish, LA",LA,19.1,18.7,19.9,20.6,20.6,21.5
2226,"Acadia Parish, LA",LA,19.1,18.7,19.9,20.6,20.6,21.5
5746,"Accomack County, VA",VA,20.2,20.5,20.5,19.6,19.5,19.8


In [12]:
# Remove duplicate rows
povDF = povDF.drop_duplicates()
povDF.head()

,County,State,2012,2013,2014,2015,2016,2017
4632,"Abbeville County, SC",SC,20.8,21.6,23.1,22.5,21.6,22.7
2290,"Acadia Parish, LA",LA,19.1,18.7,19.9,20.6,20.6,21.5
5746,"Accomack County, VA",VA,20.2,20.5,20.5,19.6,19.5,19.8
1344,"Ada County, ID",ID,12.2,13.1,12.8,12.3,12.2,11.8
1201,"Adair County, IA",IA,9.9,9.5,8.6,10.6,9.7,9.5


#### Cleaning County Poverty Data Process
1. Checked CSV to see if there were any NULL values that existed
2. Since there were only a handfull of NULL values the decision was made to drop those rows
3. The DataFrame was sorted by County to allow for a quick quality check of the data, which showed all rows were duplicated
4. Finaly, we removed these duplicate rows

## Clean FRED Poverty Data by State

In [13]:
# Replace State Initials with full state name

# Create a dictionary for each of the state abbreviations so this can be merged with CDC data flawlessly
abbrevToFullName = {'AL': 'Alabama', 'MT': 'Montana' , 'AK': 'Alaska', 'NE': 'Nebraska',
                    'AZ': 'Arizona', 'NV': 'Nevada','AR': 'Arkansas', 'NH': 'New Hampshire',
                    'CA': 'California', 'NJ': 'New Jersey', 'CO': 'Colorado', 'NM': 'New Mexico',
                    'CT': 'Connecticut', 'NY': 'New York', 'DE': 'Delaware', 'NC': 'North Carolina',
                    'FL': 'Florida', 'ND': 'North Dakota', 'GA': 'Georgia', 'OH': 'Ohio',
                    'HI': 'Hawaii', 'OK': 'Oklahoma', 'ID': 'Idaho', 'OR': 'Oregon', 'IL': 'Illinois',
                    'PA': 'Pennsylvania', 'IN': 'Indiana', 'RI': 'Rhode Island', 'IA': 'Iowa', 'SC': 'South Carolina',
                    'KS': 'Kansas', 'SD': 'South Dakota', 'KY': 'Kentucky', 'TN': 'Tennessee', 'LA': 'Louisiana',
                    'TX': 'Texas', 'ME': 'Maine', 'UT': 'Utah', 'MD': 'Maryland', 'VT': 'Vermont', 
                    'MA': 'Massachusetts', 'VA': 'Virginia', 'MI': 'Michigan', 'WA': 'Washington',
                    'MN': 'Minnesota', 'WV': 'West Virginia', 'MS': 'Mississippi', 'WI': 'Wisconsin', 
                    'MO': 'Missouri', 'WY': 'Wyoming', 'DC': 'District of Columbia'}

# Add a space in front of each key in the dictionary, original CSV has a SPACE before each abbreviation
abbrevToFullNameWithSpaces = {(" " + abbrev):full for abbrev, full in abbrevToFullName.items()}

# Show adjusted dictionary with spaces added
print(abbrevToFullNameWithSpaces)

{' AL': 'Alabama', ' MT': 'Montana', ' AK': 'Alaska', ' NE': 'Nebraska', ' AZ': 'Arizona', ' NV': 'Nevada', ' AR': 'Arkansas', ' NH': 'New Hampshire', ' CA': 'California', ' NJ': 'New Jersey', ' CO': 'Colorado', ' NM': 'New Mexico', ' CT': 'Connecticut', ' NY': 'New York', ' DE': 'Delaware', ' NC': 'North Carolina', ' FL': 'Florida', ' ND': 'North Dakota', ' GA': 'Georgia', ' OH': 'Ohio', ' HI': 'Hawaii', ' OK': 'Oklahoma', ' ID': 'Idaho', ' OR': 'Oregon', ' IL': 'Illinois', ' PA': 'Pennsylvania', ' IN': 'Indiana', ' RI': 'Rhode Island', ' IA': 'Iowa', ' SC': 'South Carolina', ' KS': 'Kansas', ' SD': 'South Dakota', ' KY': 'Kentucky', ' TN': 'Tennessee', ' LA': 'Louisiana', ' TX': 'Texas', ' ME': 'Maine', ' UT': 'Utah', ' MD': 'Maryland', ' VT': 'Vermont', ' MA': 'Massachusetts', ' VA': 'Virginia', ' MI': 'Michigan', ' WA': 'Washington', ' MN': 'Minnesota', ' WV': 'West Virginia', ' MS': 'Mississippi', ' WI': 'Wisconsin', ' MO': 'Missouri', ' WY': 'Wyoming', ' DC': 'District of Columbi

In [14]:
# Map full state name into the CSV by pairing full state name from dictionary with the state abbreviation
povDF['Full State Name'] = povDF['State'].map(abbrevToFullNameWithSpaces)

# Preview dataframe with full state name added
povDF.head()

,County,State,2012,2013,2014,2015,2016,2017,Full State Name
4632,"Abbeville County, SC",SC,20.8,21.6,23.1,22.5,21.6,22.7,South Carolina
2290,"Acadia Parish, LA",LA,19.1,18.7,19.9,20.6,20.6,21.5,Louisiana
5746,"Accomack County, VA",VA,20.2,20.5,20.5,19.6,19.5,19.8,Virginia
1344,"Ada County, ID",ID,12.2,13.1,12.8,12.3,12.2,11.8,Idaho
1201,"Adair County, IA",IA,9.9,9.5,8.6,10.6,9.7,9.5,Iowa


In [15]:
# Set up what we want to new order to be in the revised dataframe
newOrder = ['Full State Name', 'State', 'County', '2012', '2013', '2014', '2015', '2016', '2017']

# Reorder columns in new data frame
povertyRates = povDF.reindex(columns=newOrder)

# Preview final dataFrame
povertyRates.head()

,Full State Name,State,County,2012,2013,2014,2015,2016,2017
4632,South Carolina,SC,"Abbeville County, SC",20.8,21.6,23.1,22.5,21.6,22.7
2290,Louisiana,LA,"Acadia Parish, LA",19.1,18.7,19.9,20.6,20.6,21.5
5746,Virginia,VA,"Accomack County, VA",20.2,20.5,20.5,19.6,19.5,19.8
1344,Idaho,ID,"Ada County, ID",12.2,13.1,12.8,12.3,12.2,11.8
1201,Iowa,IA,"Adair County, IA",9.9,9.5,8.6,10.6,9.7,9.5


In [16]:
# Create dataframe of median poverty rates per state
groupedPovDF = povertyRates.groupby('Full State Name')
medianPovRate2012 = groupedPovDF['2012'].median()
medianPovRate2013 = groupedPovDF['2013'].median()
medianPovRate2014 = groupedPovDF['2014'].median()
medianPovRate2015 = groupedPovDF['2015'].median()
medianPovRate2016 = groupedPovDF['2016'].median()
medianPovRate2017 = groupedPovDF['2017'].median()

groupMedianDF = pd.DataFrame({'2012': medianPovRate2012,
                              '2013': medianPovRate2013,
                              '2014': medianPovRate2014, 
                              '2015': medianPovRate2015,
                              '2016': medianPovRate2016, 
                              '2017': medianPovRate2017
                             })
# preview table of all states with median poverty rates
groupMedianDF.head()

,2012,2013,2014,2015,2016,2017
Full State Name,,,,,,
Alabama,19.8,20.90,20.5,20.2,20.30,19.30
Alaska,10.5,10.20,10.2,11.3,10.80,10.60
Arizona,20.2,19.40,19.9,19.8,20.50,19.70
Arkansas,20.4,21.50,21.1,21.0,20.60,19.80
California,15.5,16.65,16.7,16.6,16.25,15.25


In [17]:
# Export clean poverty data to csvs
povertyRates.to_csv(os.path.join('Output', 'Poverty_County_Data_Cleaned.csv'))
groupMedianDF.to_csv(os.path.join('Output', 'Poverty_States_Data_Cleaned.csv'))

#### Cleaning State Poverty Data Process
1. Since states are broken out in abbreviations, we created a dictionary that aligned state abbreviations with the full state namees, included in this step is a dictionary comprehension that adds a space to the front of each Abbreviation in the dictionary because in the original DataFrame there was a space, which needed to be added to the dictionary for easy mapping
2. Next step was to create a new column for the Full State Name, this was done by using the dictionary to map the abbreviated State codes into the new column that houses the FUll State Name
3. We then reordered the columns in the County level Poverty Dataframe for better readability
4. Finaly to complete the state dataframe we Grouped everything on the new Full State Name Column, and extracted the row that housed the county with the Median Poverty Rate for each individual state